# Classification

## Introduction

## Logistic Regression (Son)

## Decision Tree (Tu)

## Random Forest (Tho)


## Support Vector Machine (SVM) (Son)


## Naive Bayes (Tu)


## K-Nearest Neighbors (KNN) (Tho)